In [ ]:
# 관련 모듈과 라이브러리 import하기

# nn: 신경망 만들 때 쓰는 기본 도구들 (레이어, 활성화 함수 등)
import torch.nn as nn

# torch: 텐서 다루고 GPU 쓸 수 있게 해준다
import torch 

# datasets: 유명한 데이터셋 쉽게 다운받게 해줌
from torchvision import datasets

# transforms: 이미지를 모델이 먹을 수 있게 변환해줌(텐서로 변환하고 정규화)
from torchvision import transforms

# DataLoader: 데이터를 불러오는 애
from torch.utils.data import DataLoader

# numpy: 숫자 계산할 때 쓰는 라이브러리
import numpy as np

# optim: 모델을 학습시키는 최적화 알고리즘들 (경사하강법 등)
import torch.optim as optim 

# plt: 그래프 그리고 이미지 보여주는 애
import matplotlib.pyplot as plt

In [3]:
# MNIST 데이터셋 다운로드 및 불러오기

# 데이터셋을 저장할 경로 넣을 변수
download_root = 'MNIST_data/'

# 학습용 데이터셋
dataset1 = datasets.MNIST(root=download_root, 
                          train = True, 
                          # 이미지를 PyTorch FloatTensor로 변환
                          # 왜냐면 딥러닝 모델은 텐서 형태의 데이터만 알아먹음!
                          # 픽셀값을 0~1 범위로 정규화하기!
                          transform = transforms.ToTensor(),
                          download=True)

# 테스트용 데이터셋
dataset2 = datasets.MNIST(root=download_root,
                            # 학습용이 아니므로 이번에는 False!
                            train = False, 
                            transform = transforms.ToTensor(),
                            download=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



In [5]:
# 모델 생성하기

# 첫번째 모델
# fully connected layer 

class FullyConnectNN(nn.Module):
    def __init__(self):
        # 부모 클래스의 생성자를 호출 -> 부모가 가진 기능들 쓸 수 있도록!
        super(FullyConnectNN, self).__init__()
        # 레이어 정의하기
        # 28x28 크기의 이미지를 1차원 벡터(784)로 바꿈
        # 은닉층의 뉴런 개수는 512개로 설정!
        # 첫번째 은닉층
        self.fc1 = nn.Linear(28*28, 512) # 입력층 -> 은닉층
        # 두번째 은닉층
        self.fc2 = nn.Linear(512, 512) # 은닉층 -> 은닉층
        self.fc3 = nn.Linear(512, 10)  # 은닉층 -> 출력층 (0~9 숫자 분류)
        # 활성화 함수로 ReLU 사용
        self.relu = nn.ReLU()

    # forward 메서드: 데이터가 모델을 통과할 때 어떻게 변하는지 정의
    def forward(self, x):
        x1 = self.fc1(x) # 첫번째 레이어 통과
        x2 = self.relu(x1) # ReLU 활성화 함수 적용
        x3 = self.fc2(x2) # 두번째 레이어 통과
        x4 = self.relu(x3) # ReLU 활성화 함수 적용
        x5 = self.fc3(x4) # 세번째 레이어 통과
        return x5

In [6]:
#두번째 모델
# 합성곱 신경망 (Convolutional Neural Network, CNN)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째 합성곱층: 입력 채널 1개(흑백), 출력 채널 32개, 커널 크기 3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        # 두번째 합성곱층: 입력 채널 32개, 출력 채널 64개, 커널 크기 3x3
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # 세번째 합성곱층: 입력 채널 64개, 출력 채널 128개, 커널 크기 3x3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # 풀링층: 2x2 크기의 맥스 풀링
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 완전 연결층: 입력 뉴런 128*3*3, 출력 뉴런 10 (0~9 숫자 분류)
        self.fc1 = nn.Linear(128 * 3 * 3, 10)
        # 활성화 함수로 ReLU 사용
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x) # 첫번째 합성곱층 통과
        x = self.relu(x) # ReLU 활성화 함수 적용
        x = self.pool(x) # 풀링층 통과

        x = self.conv2(x) # 두번째 합성곱층 통과
        x = self.relu(x) # ReLU 활성화 함수 적용
        x = self.pool(x) # 풀링층 통과

        x = self.conv3(x) # 세번째 합성곱층 통과
        x = self.relu(x) # ReLU 활성화 함수 적용
        x = self.pool(x) # 풀링층 통과

        x = x.view(-1, 128 * 3 * 3) # fully connected layer에 넣기 위해 1차원 벡터로 쫙쫙 피기
        x = self.fc1(x) # 완전 연결층 통과
        return x

In [ ]:
# Batch 사이즈 설정하기
batch = 100
# DataLoader로 데이터셋 불러오기
train_Loader = DataLoader(dataset=dataset1, batch_size = batch, shuffle = True)
test_Loader = DataLoader(dataset=dataset2, batch_size = batch, shuffle = False)